#                       Ploting UK Local Authority Data using Plotly           #

In [ ]:
import geojson as gjs
import json
import pandas as pd
import numpy as py
import geopandas as gp
import plotly.express as px

## Defining some useful functions for understanding and simplifying GeoJSON properties ##

In [ ]:
def list_geojson_properties(geojson):
    """
    Lists the properties of all features in a GeoJSON object.

    :param geojson: GeoJSON data as a Python dictionary.
    :return: List of dictionaries, each containing the properties of one feature.
    """
    properties_list = []

    if 'features' in geojson:
        for feature in geojson['features']:
            if 'properties' in feature:
                properties_list.append(feature['properties'])

    return properties_list

In [ ]:
def simplify_geojson(geojson, precision=6, properties_to_keep=None):
    """
    Simplifies a GeoJSON object by reducing the precision of coordinates
    and removing unnecessary properties.

    :param geojson: GeoJSON data as a Python dictionary.
    :param precision: Number of decimal places for coordinates.
    :param properties_to_keep: List of property names to keep. If None, keeps all properties.
    :return: Simplified GeoJSON data as a Python dictionary.
    """
    if properties_to_keep is None:
        properties_to_keep = []

    def simplify_coordinates(coords):
        """Reduce the precision of a list of coordinates."""
        if isinstance(coords[0], list):
            # Recursive call for nested lists of coordinates
            return [simplify_coordinates(coord) for coord in coords]
        else:
            # Round coordinates to the specified precision
            return [round(coord, precision) for coord in coords]

    def simplify_feature(feature):
        """Simplify a single feature."""
        # Simplify geometry
        if 'geometry' in feature and 'coordinates' in feature['geometry']:
            feature['geometry']['coordinates'] = simplify_coordinates(feature['geometry']['coordinates'])

        # Simplify properties
        if 'properties' in feature:
            if properties_to_keep:
                feature['properties'] = {prop: feature['properties'][prop] for prop in properties_to_keep if prop in feature['properties']}
        
        return feature

    # Simplify features in the GeoJSON
    if 'features' in geojson:
        geojson['features'] = [simplify_feature(feature) for feature in geojson['features']]

    return geojson

## Getting UK Data on number of households in each postcode ##

This data comes from the 2021 census. It is given at postcode level it can be found here: https://www.nomisweb.co.uk/sources/census_2021_pc

In [ ]:
df_house = pd.read_csv('pcd_p002.csv')
df_house.head(5)

To plot as a chloropleth map we need a geoJSON. This geoJSON can be found at https://martinjc.github.io/UK-GeoJSON/. It is at the Local Authority District level so we will need to aggregate the PostCode level information to this level. 

In [ ]:
uk_jsonfile = "utl.geojson"
f = open(uk_jsonfile)
gs = f.read()
gsdata = gjs.loads(gs)
properties_list = list_geojson_properties(gsdata)
properties_list

We do not need all these properties so let's simplify to include only the ones that we want.

In [ ]:
properties_list = list_geojson_properties(gsdata)
prp  = ['LAD22CD','LAD22NM']
gs_simp = simplify_geojson(gsdata, precision=3, properties_to_keep=prp)
properties_list = list_geojson_properties(gsdata)
properties_list = list_geojson_properties(gsdata)
properties_list

Mapping table to map postcodes to LADs. This can be found in

In [ ]:
df_map1 = pd.read_csv('./GeoData/PCtoLSOA/PCD_OA21_LSOA21_MSOA21_LAD_AUG23_UK_LU.csv',encoding='latin-1',low_memory=False)

In [ ]:
df_house.columns = ['Postcode','HouseHolds']
df1 = df_house.merge(df_map1, left_on = 'Postcode', right_on = 'pcds', how = 'inner')

In [ ]:
df1.head(3)

In [ ]:
d_emp = df1.pivot_table(index = 'ladcd', values = 'HouseHolds', aggfunc='mean')
d_emp = d_emp.reset_index()
d_emp.columns = ['LAD22CD','HouseHolds']

In [ ]:
data = gp.read_file(uk_jsonfile)
dat1 = data.merge(d_emp,on = 'LAD22CD', how = 'left')
df_dat1 = dat1.dropna()

In [ ]:
df_plt = df_dat1[['LAD22CD','HouseHolds','LAD22NM']]
df_plt.head(5)

Use plotly to show the data in chloropleth format. The data is only for England and Wales, hence Scotland and Ireland are not included

In [ ]:
fig = px.choropleth_mapbox(df_plt, geojson=gs_simp, locations='LAD22CD', color='HouseHolds',featureidkey="properties.LAD22CD",
                           color_continuous_scale="Viridis", 
                           range_color=(0, 30),
                           mapbox_style="carto-positron",
                           center  = {"lat": 56, "lon" : 0.12},
                           zoom=3
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()